In [1]:
!pip install nes_py
!pip install gym_super_mario_bros
!pip install tensordict
!pip install torchrl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for nes_py: filename=nes_py-8.2.1-cp310-cp310-linux_x86_64.whl size=48172 sha256=2f4d42481b09620e4b10b5529746540eeec736b1e7a9ec410ab099a9975c9f64
  Stored in directory: /root/.cache/pip/wheels/34/a7/d5/9aa14b15df740a53d41f702e4c795731b6c4da7925deb8476c
Successfully built nes_py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.1/336.1 kB 5.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 988.0/988.0 kB 13.9 MB/s eta 0:00:0000:010:01


In [2]:
import numpy as np
import nes_py
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from IPython import display
import time
from PIL import Image
from IPython.display import clear_output

import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

from tensordict import TensorDict
from torchrl.data import TensorDictReplayBuffer, LazyMemmapStorage


from gym import Wrapper
from gym.wrappers import GrayScaleObservation,ResizeObservation,FrameStack
import cv2

from tqdm import tqdm

In [3]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

In [4]:
custom_movement=[['right','A',"B"],['right','B']]

In [5]:
env=gym_super_mario_bros.make('SuperMarioBros-1-1-v0',apply_api_compatibility=True,render_mode='rgb_array')
env=JoypadSpace(env,custom_movement)

/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [6]:
# terminated,truncated=True,True
# for step in range(2000):
#     if terminated or truncated:
#         state = env.reset()[0]  
#     state, reward, terminated, truncated, info = env.step(1)
#     screen = env.render()

#     image = Image.fromarray(screen)
#     clear_output(wait=True)

#     display.display(image)

In [7]:
class SkipFrameWrapper(Wrapper):
    def __init__(self,env,skip):
        super().__init__(env)
        self.skip=skip
    
    def step(self,action):
        total_reward=0.0
        term,trunc=False,False
        for i in range(self.skip):
            next_state,reward,term,trunc,info=self.env.step(action)
            total_reward+=reward
            if term or trunc:
                break
        return next_state,total_reward,term,trunc,info
    

def get_env(custom_movement):
    # default frame size is 240*256
    # mario renders 60 frames per sec
    # current 10 steps per sec 
    # Currently using 1/2 sec of input state skip=6,stack=5,could change based on training
    env=gym_super_mario_bros.make('SuperMarioBros-1-1-v0',apply_api_compatibility=True,render_mode='rgb_array')
    env=JoypadSpace(env,custom_movement)
    env=ResizeObservation(env,shape=60) 
    env=GrayScaleObservation(env)
    env=SkipFrameWrapper(env,skip=5)
    env=FrameStack(env,num_stack=3,lz4_compress=True) 
    return env

In [8]:
class Swish(nn.Module):
    def forward(self,x):
        return x*torch.sigmoid(x)

class DQN(nn.Module):
    #Could try freezing just linear layer
    def __init__(self,input_shape,mario_actions,conv_layer_size=[32,16,16],fc_size=[128,128],freeze_complete_net=False):
        super().__init__()
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.to(self.device)

        self.conv_layers=nn.Sequential(
            nn.Conv2d(input_shape[0],conv_layer_size[0],kernel_size=8,stride=5),
            Swish(),
            nn.Conv2d(conv_layer_size[0],conv_layer_size[1],kernel_size=5,stride=3),
            Swish(),
            nn.Conv2d(conv_layer_size[1],conv_layer_size[2],kernel_size=3,stride=2),
            Swish(),
        )

        self.conv_out_size = self.get_conv_output_size(input_shape)

        self.network = nn.Sequential(
            self.conv_layers,
            nn.Flatten(),
            nn.Linear(self.conv_out_size,fc_size[0]),
            Swish(),
            nn.Linear(fc_size[0],fc_size[1]),
            Swish(),
            nn.Linear(fc_size[1],mario_actions)
        )
        
        if freeze_complete_net:
            self.freeze_net()            

        
    def get_conv_output_size(self,shape):
        # for future me remember nn require batched input thats why 1 is imp
        dummy_inp=torch.zeros(1,*shape)
        output=self.conv_layers(dummy_inp)
        dim=output.size()
        return int(np.prod(dim))

    def forward(self, x):
        return self.network(x)

    def freeze_net(self,complete=False):        
        for p in self.network.parameters():
            p.requires_grad = False
            
        for p in self.conv_layers.parameters():
            p.requires_grad = False

In [9]:
#Future me sorry for this big class i am just a human afterall
class Mario:
    def __init__(self,move_count,input_shape,
                 lr=.002,gamma=.99,
                 epsilon=1.0,eps_min=0.1,eps_decay=1-5e-6, 
                 replay_buffer_capacity=1e5,batch_size=64, 
                 sync_network_rate=1e4):

        self.move_count=move_count
        self.steps_so_far=0
        
        self.lr=lr
        self.gamma=gamma
        self.epsilon=epsilon
        self.eps_decay=eps_decay
        self.eps_min=eps_min
        self.batch_size=batch_size
        self.replay_buffer_capacity=replay_buffer_capacity
        self.sync_network_rate=sync_network_rate

        self.online_net=DQN(input_shape,move_count)
        self.target_net=DQN(input_shape,move_count,freeze_complete_net=True)
        
        self.optimizer=torch.optim.Adam(self.online_net.parameters(), lr=self.lr)
        self.loss_huber=nn.SmoothL1Loss()
        
        storage=LazyMemmapStorage(replay_buffer_capacity)
        self.replay_buffer = TensorDictReplayBuffer(storage=storage)

    def sync_networks(self):
        #another naming genius by me (laughing emoji)
        time_has_come=((self.steps_so_far+1)%self.sync_network_rate)==0
        if time_has_come:
            cur_online_state=self.online_net.state_dict()
            self.target_net.load_state_dict(cur_online_state)


    def choose_action(self, observation):
        take_random_action=np.random.random()<self.epsilon
        if take_random_action:
            random_action=np.random.randint(self.move_count)
            return random_action
        
        observation=np.array(observation)
        observation=torch.tensor(observation,dtype=torch.float32)
        observation=observation.unsqueeze(0)
        
        best_action=self.online_net(observation).argmax().item()
        return best_action        

    def add_to_replay_buffer(self,cur_state,next_state,action,reward,term,trunc):
        self.replay_buffer.add(TensorDict({ "state": torch.tensor(np.array(cur_state),dtype=torch.float32), 
                                            "next_state": torch.tensor(np.array(next_state),dtype=torch.float32), 
                                            "action": torch.tensor(action),
                                            "reward": torch.tensor(reward), 
                                            "term": torch.tensor(term),
                                            "trunc": torch.tensor(trunc)
                                          }, batch_size=[]))
        
    def save_model(self, path):
        torch.save(self.online_net.state_dict(), path)

    def load_model(self, path):
        self.online_net.load_state_dict(torch.load(path))
        self.target_net.load_state_dict(torch.load(path))


    def learn(self):
        cant_learn=len(self.replay_buffer)<self.batch_size 

        if cant_learn:
        # us bro us
            return

        self.sync_networks()

        self.optimizer.zero_grad()

        samples=self.replay_buffer.sample(self.batch_size)
        keys=("state","action","reward","next_state","term","trunc")

        states,actions,rewards,next_states,term,trunc=[samples[key] for key in keys]

        predicted_q_values=self.online_net(states)
        predicted_q_values=predicted_q_values[np.arange(self.batch_size),actions.squeeze()]

        target_q_values=self.target_net(next_states).max(dim=1)[0]
        target_q_values=rewards+self.gamma*target_q_values*(1-(term.float()+trunc.float()))

        loss_huber=self.loss_huber(predicted_q_values,target_q_values)
        loss_huber.backward()
        self.optimizer.step()

        self.steps_so_far += 1
        self.epsilon=max(self.epsilon*self.eps_decay,self.eps_min)

In [20]:
custom_movement=[['right','A',"B"],['right','B']]
env=get_env(custom_movement)

/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [21]:
mario=Mario(input_shape=env.observation_space.shape,move_count=env.action_space.n,eps_min=0,epsilon=0)

In [ ]:
EPISODES=5000

In [ ]:
disp=False
for i in tqdm(range(EPISODES)):
    term,trunc=False,False
    state=env.reset()[0]
    while not (term or trunc):
        move=mario.choose_action(state)
        new_state,reward,term,trunc,info=env.step(move)
        mario.add_to_replay_buffer(state,new_state,move,reward,term,trunc)
        mario.learn()
        state = new_state
        if disp:
            screen = env.render()

            image = Image.fromarray(screen)
            clear_output(wait=True)

            display.display(image)

In [ ]:
mario.save_model("My_Mario")